# Code

In [2]:
# Imports and setup model
import os
import openai
import json
import configobj
import tiktoken
from time import sleep

In [3]:
# Setup env vars and constants
config = configobj.ConfigObj('.env')
openai.api_key = config["OPENAI_API_KEY"]

api_model = "gpt-4"
dimensions = 1536
docs_dir = 'docs/'

In [4]:
#Setup Tokenizer
tokenizer_name = tiktoken.encoding_for_model(api_model)
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)


In [5]:
# load docs
texts = []
docs_by_name = {}

for doc in os.listdir(docs_dir):
    with open(docs_dir + doc, 'r') as f:
        text = f.read()
        texts.append(text)
        docs_by_name[doc[0:-3]] = text

In [6]:
state = ""
with open('state.json') as f:
    state = f.read()

In [76]:
topLevel = ""
with open(f'{docs_dir}topLevelInfo.md') as f:
    topLevel = f.read()

print(tiktoken_len(topLevel))

1066


In [79]:
query = "Size the warehouses by capacity"

# Run Query through gpt4

Note: Currently using gpt-4(8k) as gpt-4-32k hasn't been publicly released yet and gpt-3.5-turbo-16k isn't great

In [80]:
completion = openai.ChatCompletion.create(
  model=api_model,
  temperature=0.1,
  messages=[
    {"role": "system", "content": "Using the given documentation delimited by triple quotes, respond to user requests with nothing but the name of the top-level key that must be edited in order to fufill the request surrounded by single quotes"},
    {"role": "user", "content": '"""' + topLevel + '""" \n\n' + 'request: ' + query},
  ]
)
current_key = completion.choices[0].message.content.replace("'", '')

print(current_key)

maps


In [81]:
encoded_data = json.dumps(json.loads(state)[current_key])
max_size = False
chunks = [None]
if tiktoken_len(encoded_data) > 7000:
    max_size = True
    key_dict = json.loads(state)[current_key]
    chunks[-1] = {i:key_dict[i] for i in key_dict if i!='data'}
    chunks[-1]['data'] = {}
else:
    chunks = [encoded_data]
print(tiktoken_len(json.dumps(chunks)))
if max_size:
    for key, value in key_dict['data'].items():
        if tiktoken_len(json.dumps(chunks[-1]) + json.dumps({key: value})) > 7000:
            chunks.append({i:key_dict[i] for i in key_dict if i!='data'})
            chunks[-1]['data'] = {key: value}
        else:
            chunks[-1]['data'][key] = value      
    for idx, chunk in enumerate(chunks):
        chunks[idx] = json.dumps(chunk)

for data in chunks:
    print(tiktoken_len(data))

print(len(chunks))

496
470
1


In [ ]:
chunks[0]

'{"name": "nodes", "types": {"nodeTypeRT": {"name": "Rail Terminals", "colorByOptions": {"active": {"false": "rgb(150,150,150)", "true": "rgb(8,255,8)"}, "throughput": {"min": 0, "max": 2000, "startGradientColor": {"dark": "rgb(150,150,150)", "light": "rgb(150,150,150)"}, "endGradientColor": {"dark": "rgb(8,255,8)", "light": "rgb(8,255,8)"}}}, "colorBy": "active", "minSizeRange": 0, "sizeByOptions": {"capacity": {"min": 0, "max": 2000}, "throughput": {"min": 0, "max": 2000}}, "sizeBy": "throughput", "startSize": "10px", "endSize": "25px", "icon": "MdDirectionsRailwayFilled", "props": {"capacity": {"name": "Throughput Capacity", "type": "num", "enabled": true, "help": "Max. number of units per year", "numberFormat": {"unit": "units"}}, "active": {"name": "Rail terminal active", "type": "toggle", "value": true, "enabled": true, "help": "Enable / disable rail terminal"}, "throughput": {"name": "Facility Throughput", "type": "num", "enabled": false, "help": "Number of units per year", "num

In [82]:
for data in chunks:
  completion = openai.ChatCompletion.create(
    model=api_model,
    temperature=0.1,
    messages=[
      {"role": "system", "content": "Using the given documentation delimited by triple quotes and current api state encoded in JSON, respond to user requests with the path(s) and value(s) you would modify in the api to achieve the users desired action in the format \n```{\"path\": example.path.here, \"value\": value}```. If the user request isn't possible with the given data respond with 'Not possible'"},
      {"role": "user", "content": '"""' + docs_by_name[current_key] + '""" \n\n' + data + '\n\nrequest: ' + query},
    ]
  )

  print(completion.choices[0].message.content)
  mutate_command = completion.choices[0].message.content
  if not 'Not possible' in mutate_command:
    print('Command output!')
    break

{"path": "data.map1.legendGroups.Network Facilities.nodes.nodeTypeWH.sizeBy", "value": "capacity"}
Command output!
